## Tests methods for compressing readings sent through Notecard

In [1]:
import pickle
import random
import string
import time
import ast

In [2]:
sensor_ids = []
sensor_ct = 15
for i in range(sensor_ct):
    sensor_ids.append(''.join(random.choices(string.ascii_lowercase, k=20)))
sensor_ids

['kmyofckxvbfkopxedsdq',
 'lmjdakrydmagxxprzhcd',
 'qyjirhmqjzwrrhrvnpxo',
 'ihronsdyspufcygjamkh',
 'omybzhrbjblwtixyzxyg',
 'vkqknluftkcqchzzwrte',
 'nbwiizauruzttmggeavf',
 'dzcnotpoonlkmjnsfjhd',
 'krtjqfwgofgvoafmhigs',
 'eptlfexqfhuwbmnbiamb',
 'xjafbiuhndoiibiewdld',
 'ufmnbmhgylyiouthicrz',
 'tavppdrolskzkszkidoe',
 'cvbwxdsvspsigttkkvit',
 'dnvzojhnbqcnmilxixoh']

In [3]:
readings = []
ts = round(time.time(), 1)
for tstep in range(6):
    t = ts + tstep * 600
    for sensor in sensor_ids:
        val = round(random.random(), 6)
        readings.append( (t, sensor, val))
readings

[(1671044959.1, 'kmyofckxvbfkopxedsdq', 0.278581),
 (1671044959.1, 'lmjdakrydmagxxprzhcd', 0.818572),
 (1671044959.1, 'qyjirhmqjzwrrhrvnpxo', 0.607011),
 (1671044959.1, 'ihronsdyspufcygjamkh', 0.675082),
 (1671044959.1, 'omybzhrbjblwtixyzxyg', 0.85945),
 (1671044959.1, 'vkqknluftkcqchzzwrte', 0.559337),
 (1671044959.1, 'nbwiizauruzttmggeavf', 0.772609),
 (1671044959.1, 'dzcnotpoonlkmjnsfjhd', 0.671232),
 (1671044959.1, 'krtjqfwgofgvoafmhigs', 0.327291),
 (1671044959.1, 'eptlfexqfhuwbmnbiamb', 0.177227),
 (1671044959.1, 'xjafbiuhndoiibiewdld', 0.648947),
 (1671044959.1, 'ufmnbmhgylyiouthicrz', 0.301236),
 (1671044959.1, 'tavppdrolskzkszkidoe', 0.178509),
 (1671044959.1, 'cvbwxdsvspsigttkkvit', 0.873062),
 (1671044959.1, 'dnvzojhnbqcnmilxixoh', 0.34815),
 (1671045559.1, 'kmyofckxvbfkopxedsdq', 0.609853),
 (1671045559.1, 'lmjdakrydmagxxprzhcd', 0.814025),
 (1671045559.1, 'qyjirhmqjzwrrhrvnpxo', 0.509386),
 (1671045559.1, 'ihronsdyspufcygjamkh', 0.354936),
 (1671045559.1, 'omybzhrbjblwtixy

In [4]:
data = str(readings).encode('utf-8')
print('Length as uncompressed text', len(data))

Length as uncompressed text 4489


### Remember that we then have to convert compressed bytes to Base 64
Compression ratio will be lowered to 0.75 x value shown here.

In [5]:
import bz2
# Compress the string representation
c = bz2.compress(data)
print('bz2 compression ratio', len(data) / len(c))

bz2 compression ratio 4.6518134715025905


In [6]:
pickle_data = pickle.dumps(readings, pickle.HIGHEST_PROTOCOL)
print('pickle alone', len(data) / len(pickle_data))

pickle alone 1.942449156209433


In [7]:
print('pickle with bz2 compression', len(data) / len(bz2.compress(pickle_data)))

pickle with bz2 compression 3.014775016789792


Looks like BZ2 compression of the string representation of the Readings Array is best.

Will then need to convert to Base64 and could assign to the Payload key of the Note.

## Test of compression, decompression

In [8]:
import base64

data = bz2.compress(str(readings).encode('utf-8'))
sdata = base64.b64encode(data).decode('utf-8')
len(str(readings)), len(data), len(sdata)

(4489, 965, 1288)

In [9]:
print(sdata)

QlpoOTFBWSZTWSckXaUABUBbgEAAQOV/4AAKP///8FAE7HQABsPIqUgkoJinqAAZPUDQ9IAYaGQ0yaAYhppo0NGDTRoClUADTNqQAACKSBqM1T0j9RoagADRoIqqeyU9T0xoynlTNQAABOCEgDdwZVl1rHUwsIDWVP0ecpOrOROTXyg57XCebeFuddHXulsuOx4Vx11yCOOuC69V93zMzMw9EBgw9e4+4OfZx7P7H0vtiZlGITSymnm3qIM+ny3N1OGrNw80FRsIWXmptVdw0PaLLRFNUFOYing4CKQqJkIptlflleCu8PoRtxoYMwrdMPMfKMB53JmrVGoNjc0MGYKwwZh05u9u5arasYXIt2Zq3fQYMwy0upyc88tw+CY4zTxi8vdeH1G9M9ysVfDI2oiM7XQqOgnhsaKulcOam+OYFEUbmFfLReway0BudEpmbst+25tPkXMxNvY2bGdsJHprrFszPsYmZnGF2ZssxaZml7xMzJ3AAmDCtmaAd5Wn3lj5R2lEAjOeqeKh4Ik2b13xmaUzNUvkdUHcq6IgxCeIMJ08JCFEIukkiIhAqExeEgnhIF1CLpJAmIQh4dQnSGq8i3tHl252ZL7gkAAkQdu73rs9zxrjpchgzAhMzYkHhYbe1JEOLNmaZgzBx1ScUdeszSTOY7szc+RKhyE7zQAFcnrZcb04IrqviR3WqCnX5W82eLkWswX6Qfnx9DBmGT4ujx6p4pP5fkVNBw9oPpzgSQQQQQQSSQSQQdvgq68RGyHLReK4XgVBdhPFQVoqCgL2LAURXi9DsG0f0YBvHqPoPsMxgaD+DiPqNw9A6TYJEmTQaDiOAzGg/I/o1HqNg3jv0iIiIiIjoNw2jsHcO8f4cB6jUZDoOQ/w3jeDaPaNB5jIajQGgzHQcjiREMPMeQ8BqPQO4wHUbxyGI5DuMxiOY5DAbhoNhkOwyGY6jYcR6xoOg3DUYjMdw4jtGwZjwGBkNRkOg5jcMxoNg6Dg

In [1]:
sdata = "QlpoOTFBWSZTWX2YBkoAAfPfgFAAQOd/4BgAAAqKI98AMAF7QAYYJgTAQ0ZNMBplIAAaAAAIlCYRJpPUHqekegnlNA4QUqPt3H4G41TAAdoo/rXSUnT80rHgouBRvxTi5IkMoSSAEMCASA9gzXBmb7CG9pJJJJMjoAkoOBrWD15DUpcbg9QXmNBDbljGpsDYbDtKW2kzB7DfRkcGuY6FdvQZoVk6ivYa3efQbjbPHBuNqNeB9eYyw7xODBwNXjnQvHcacsCZsM2MCWhRghClOB3bObV9a4zxbQ8bmhWBB2Nt5KCtqXplou+RuYW188GsAwxGTJfE2tjF7Na1rWta2MwhMjkdDsPt9+4+BgPcfgfA+R7iFB8jce49hDwOwhYcj/hDgfA5HyPoPIcCEGgP/F3JFOFCQfZgGSg="

In [2]:
import bz2, base64, ast
# Restore this to a readings array
data_compressed = base64.b64decode(sdata)
data = bz2.decompress(data_compressed)
readings_restored = ast.literal_eval(data.decode('utf-8'))      # eval seems to work with bytes
readings_restored

[(1671152998.0, '12.B169C3000000', 0.0),
 (1671152999.0, 'mtn-pi_version', 3.8),
 (1671152700.5, '28.64104C050000', 68.203),
 (1671152999.0, 'mtn-pi_uptime', 7262.0),
 (1671152701.5, 'mtn-pi_cpu_temp', 59.282),
 (1671153598.0, '12.B169C3000000', 0.0),
 (1671153599.1, 'mtn-pi_version', 3.8),
 (1671153300.5, '28.64104C050000', 67.786),
 (1671153599.1, 'mtn-pi_uptime', 7862.0),
 (1671153301.5, 'mtn-pi_cpu_temp', 59.144),
 (1671154198.0, '12.B169C3000000', 0.0),
 (1671154199.0, 'mtn-pi_version', 3.8),
 (1671153900.4, '28.64104C050000', 67.944),
 (1671154199.0, 'mtn-pi_uptime', 8462.0),
 (1671153901.5, 'mtn-pi_cpu_temp', 59.116),
 (1671154798.0, '12.B169C3000000', 0.0),
 (1671154799.0, 'mtn-pi_version', 3.8),
 (1671154500.4, '28.64104C050000', 68.536),
 (1671154799.0, 'mtn-pi_uptime', 9062.0),
 (1671154501.5, 'mtn-pi_cpu_temp', 59.185),
 (1671155398.0, '12.B169C3000000', 0.0),
 (1671155399.0, 'mtn-pi_version', 3.8),
 (1671155100.5, '28.64104C050000', 69.159),
 (1671155399.0, 'mtn-pi_uptime'

## Another Compression Method

Use a 1-byte number to identify the sensor ID (limit of 256 IDs in one batch of readings),
express timestamp as a delta from a base timestamp.  The delta would be expressed in tenths
of a second; since less than one hour of data is tranmitted, we only need the numbers
0 - 36,000 to epress the delta.  The value would be epressed as a 4-byte, single-precision
floating point value.

Total record size is therefore 7 bytes.  Just build a large bytearray of 7-byte records.
Compress with bz2 and then encode as a base64 string.

In [74]:
raw_str_data = str(readings).encode('utf-8')
len(raw_str_data)

4494

In [75]:
recs = bytearray()
recs.append(4)
recs.append(5)
recs += b'abc'
recs

bytearray(b'\x04\x05abc')

In [86]:
ts_arr, id_arr, val_arr = zip(*readings)

# create dictionary of all unique sensor IDs mapped to a sensor integer
ids_uniq = list(set(id_arr))
sensor_map = dict(zip(ids_uniq, range(len(ids_uniq))))

# get the minimum ts to use as a base
ts_base = min(ts_arr)


In [81]:
# need specify Byte order to avoid automatic padding
import struct
rec = struct.pack('<HBd', 36000, 255, 2.434e-5)
struct.unpack('<HBd', rec)

(36000, 255, 2.434e-05)

In [82]:
# Need to use a Double for the val field in order to accommodate counters.
recs = b''
for ts, sensor_id, val in readings:
    rec = struct.pack(
        '<HBd',                       # need specify Byte order to avoid automatic padding
        int((ts - ts_base) * 10), 
        sensor_map[sensor_id], 
        val)
    recs += rec
len(recs)

990

In [83]:
# Compression **increases** the size of the byte array!  It was already highly compressed.
# This is worse than just compressing the string representation of the readings array,
# and more complicated.
len(bz2.compress(recs)), len(recs)

(1090, 990)

In [93]:
# Curious whether substituting sensor ID integers helps the string compression 
# in the original method
id_int_arr = [sensor_map[id] for id in id_arr]
readings_2 = list(zip(ts_arr, id_int_arr, val_arr))
print('Integer IDs', len(bz2.compress(str(readings_2).encode('utf-8'))))
print('String IDs', len(bz2.compress(str(readings).encode('utf-8'))))

# ***BUT*** When you add in the Sensor Map that you need to send along with this payload
# The total bytes were 1189 bytes, as opposed to just compressing the unaltered string 
# version of the readings array, which comes out to 1284 bytes.  So, the savings are really
# very little and not worth the additional complexity.

Integer IDs 565
String IDs 961


In [94]:
# What about also substituting ts deltas in tenths
ts_delta_arr = [int(10 * (ts - ts_base)) for ts in ts_arr]
readings_3 = list(zip(ts_delta_arr, id_int_arr, val_arr))
print('Integer IDs + Delta ts', len(bz2.compress(str(readings_3).encode('utf-8'))))
# Very little additional compression


Integer IDs + Delta ts 540


In [98]:
# How does this algorithm work for a small array:
d = {
	"storeKey": "124343abc",
	"readings": [
		[6542342.2, "abc123", 23.4],
		[6542344.8, "xyz_456", 33.4]
	]
}
len(str(d['readings']))

59

In [99]:
small_rd = [
		[6542342.2, 0, 23.4],
		[6542344.8, 1, 33.4]
	]
print('Small Compressed', len(bz2.compress(str(small_rd).encode('utf-8'))))

# not terrible, except do need to add the sensor map into the body.


Small Compressed 65


In [100]:
# Try lzma with Integer Sensor ID reading array
import lzma
print('Integer IDs', len(lzma.compress(str(readings_2).encode('utf-8'))))

# a little worse than bz2.


Integer IDs 616


### Experiments

In [112]:
from pathlib import Path
import sys
stg_path = Path('/boot/pi_logger/settings.py')
sys.path.insert(0, str(stg_path.parent))
sys.path

['/boot/pi_logger',
 '/home/alan/notecard-server/test',
 '/home/alan/anaconda3/lib/python39.zip',
 '/home/alan/anaconda3/lib/python3.9',
 '/home/alan/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/home/alan/anaconda3/lib/python3.9/site-packages',
 '/home/alan/anaconda3/lib/python3.9/site-packages/IPython/extensions',
 '/home/alan/.ipython']